In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# pd.options.plotting.backend = "plotly"

# import dash
# import dash_core_components as dcc
# import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.tree import export_graphviz
from graphviz import Source

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier, plot_importance

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.ensemble import VotingClassifier, RandomForestClassifier

import time
import pandas_profiling

Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 255 ('font.family:AppleGothic')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 256 ('font.size:12')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 370 ('axes.titlesize:     large   # fontsize of the axes title')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 376 ('axes.labelsize:     medium  # fontsize of the x any y labels')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 410 ('axes.unicode_minus: True  # use Unicode for the minus symbol rather than hyphen.  See')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib

In [2]:
pokemon = pd.read_csv('./pokemon_df_diff.csv')
# pokemon_df.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace= True)

In [3]:
pokemon

,Unnamed: 0,Unnamed: 0.1,#_x,First_pokemon,Second_pokemon,Winner_mark,Type 1_first,Type 2_first,HP_first,Attack_first,...,Sp. Def_second,Speed_second,Generation_second,Legendary_second,HP_diff,Attack_diff,Defense_diff,Sp. Atk_diff,Sp. Def_diff,Speed_diff
0,0,0,266,266,298,2,15,10,50,64,...,40,60,3,0,-20,-6,10,-15,10,-19
1,1,1,702,702,701,2,9,5,91,90,...,90,108,5,1,0,-39,-18,18,39,0
2,2,2,191,191,668,2,4,7,55,40,...,95,40,5,0,-20,-35,10,-45,10,0
3,3,3,237,237,683,2,6,12,40,40,...,90,48,5,0,-37,-80,-50,10,-50,-28
4,4,4,151,151,231,1,15,18,70,60,...,230,5,2,0,50,50,-105,105,-160,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,49995,707,707,126,1,2,6,100,120,...,25,60,1,0,70,80,30,80,95,30
49996,49996,49996,589,589,664,1,10,12,60,85,...,40,60,5,0,25,30,0,-15,5,8
49997,49997,49997,303,303,368,2,17,7,60,50,...,60,90,3,0,-13,-65,40,25,10,-25
49998,49998,49998,109,109,89,1,3,12,40,30,...,55,45,1,0,15,-5,-20,-40,0,55


In [4]:
pokemon.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', '#_x', 'First_pokemon', 'Second_pokemon',
       'Winner_mark', 'Type 1_first', 'Type 2_first', 'HP_first',
       'Attack_first', 'Defense_first', 'Sp. Atk_first', 'Sp. Def_first',
       'Speed_first', 'Generation_first', 'Legendary_first', '#_y',
       'Type 1_second', 'Type 2_second', 'HP_second', 'Attack_second',
       'Defense_second', 'Sp. Atk_second', 'Sp. Def_second', 'Speed_second',
       'Generation_second', 'Legendary_second', 'HP_diff', 'Attack_diff',
       'Defense_diff', 'Sp. Atk_diff', 'Sp. Def_diff', 'Speed_diff'],
      dtype='object')

In [5]:
assert len(pokemon.columns) == len (set(pokemon.columns))

In [6]:
cat_cols = ['#_x', 'First_pokemon', 'Second_pokemon', 
            'Legendary_first', 'Legendary_second']
conti_cols = ['HP_first', 'Attack_first',
              'Defense_first', 'Sp. Atk_first', 'Sp. Def_first', 'Speed_first',
              'Generation_first', 
              'HP_second', 'Attack_second', 'Defense_second',
              'Sp. Atk_second', 'Sp. Def_second', 'Speed_second', 'Generation_second']
diff = ['HP_diff', 'Attack_diff',
       'Defense_diff', 'Sp. Atk_diff', 'Sp. Def_diff', 'Speed_diff']

y = ['Winner_mark']

In [ ]:
X = pokemon['Speed_first', 'Speed_second', 'Legendary_first','Legendary_second', 'Speed_diff']
y = pokemon[y]
y = np.ravel (y, order = "c")

In [7]:
X = pokemon[cat_cols + conti_cols]
y = pokemon[y]

In [ ]:
y = np.ravel (y, order = "c")
y.shape

In [8]:
X.columns

Index(['#_x', 'First_pokemon', 'Second_pokemon', 'Legendary_first',
       'Legendary_second', 'HP_first', 'Attack_first', 'Defense_first',
       'Sp. Atk_first', 'Sp. Def_first', 'Speed_first', 'Generation_first',
       'HP_second', 'Attack_second', 'Defense_second', 'Sp. Atk_second',
       'Sp. Def_second', 'Speed_second', 'Generation_second'],
      dtype='object')

In [9]:
X

,#_x,First_pokemon,Second_pokemon,Legendary_first,Legendary_second,HP_first,Attack_first,Defense_first,Sp. Atk_first,Sp. Def_first,Speed_first,Generation_first,HP_second,Attack_second,Defense_second,Sp. Atk_second,Sp. Def_second,Speed_second,Generation_second
0,266,266,298,0,0,50,64,50,45,50,41,2,70,70,40,60,40,60,3
1,702,702,701,1,1,91,90,72,90,129,108,5,91,129,90,72,90,108,5
2,191,191,668,0,0,55,40,85,80,105,40,2,75,75,75,125,95,40,5
3,237,237,683,0,0,40,40,40,70,40,20,2,77,120,90,60,90,48,5
4,151,151,231,0,0,70,60,125,115,70,55,1,20,10,230,10,230,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,707,707,126,1,0,100,120,100,150,120,90,5,30,40,70,70,25,60,1
49996,589,589,664,0,0,60,85,40,30,45,68,5,35,55,40,45,40,60,5
49997,303,303,368,0,0,60,50,100,85,70,65,3,73,115,60,60,60,90,3
49998,109,109,89,0,0,40,30,50,55,55,100,1,25,35,70,95,55,45,1


In [10]:
# X['First_pokemon'] == 

In [11]:
X['Total_first'] = X['HP_first'] + X['Attack_first'] + X['Defense_first'] + X['Sp. Atk_first'] + X['Sp. Def_first']+X['Speed_first']

<ipython-input-11-e0c4cd716539>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Total_first'] = X['HP_first'] + X['Attack_first'] + X['Defense_first'] + X['Sp. Atk_first'] + X['Sp. Def_first']+X['Speed_first']


In [12]:
X['Total_second'] = X['HP_second'] + X['Attack_second'] + X['Defense_second'] + X['Sp. Atk_second'] + X['Sp. Def_second']+X['Speed_second']

<ipython-input-12-719a01743d9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Total_second'] = X['HP_second'] + X['Attack_second'] + X['Defense_second'] + X['Sp. Atk_second'] + X['Sp. Def_second']+X['Speed_second']


In [13]:
X

,#_x,First_pokemon,Second_pokemon,Legendary_first,Legendary_second,HP_first,Attack_first,Defense_first,Sp. Atk_first,Sp. Def_first,...,Generation_first,HP_second,Attack_second,Defense_second,Sp. Atk_second,Sp. Def_second,Speed_second,Generation_second,Total_first,Total_second
0,266,266,298,0,0,50,64,50,45,50,...,2,70,70,40,60,40,60,3,300,340
1,702,702,701,1,1,91,90,72,90,129,...,5,91,129,90,72,90,108,5,580,580
2,191,191,668,0,0,55,40,85,80,105,...,2,75,75,75,125,95,40,5,405,485
3,237,237,683,0,0,40,40,40,70,40,...,2,77,120,90,60,90,48,5,250,485
4,151,151,231,0,0,70,60,125,115,70,...,1,20,10,230,10,230,5,2,495,505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,707,707,126,1,0,100,120,100,150,120,...,5,30,40,70,70,25,60,1,680,295
49996,589,589,664,0,0,60,85,40,30,45,...,5,35,55,40,45,40,60,5,328,275
49997,303,303,368,0,0,60,50,100,85,70,...,3,73,115,60,60,60,90,3,430,458
49998,109,109,89,0,0,40,30,50,55,55,...,1,25,35,70,95,55,45,1,330,325


In [14]:
X.columns

Index(['#_x', 'First_pokemon', 'Second_pokemon', 'Legendary_first',
       'Legendary_second', 'HP_first', 'Attack_first', 'Defense_first',
       'Sp. Atk_first', 'Sp. Def_first', 'Speed_first', 'Generation_first',
       'HP_second', 'Attack_second', 'Defense_second', 'Sp. Atk_second',
       'Sp. Def_second', 'Speed_second', 'Generation_second', 'Total_first',
       'Total_second'],
      dtype='object')

In [15]:
# train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=1)

# train / validation split
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train, random_state=1)

y_train.shape, y_train.shape, X_test.shape, y_test.shape

((37500, 1), (37500, 1), (12500, 21), (12500, 1))

In [16]:
assert len(X_train.columns) ==len(set(X_train.columns))

In [17]:
y_train

,Winner_mark
47852,1
35448,2
38630,1
21791,1
43649,1
...,...
5327,2
1282,2
6779,2
34830,1


In [33]:
y_train = np.ravel (y_train, order = "c")
y_train.shape

(37500,)

## GridSearchCV - pipeline

#### SVC

In [18]:
pipeline_svc = make_pipeline(StandardScaler(), SVC())
pipeline_svc.steps # class 이름 소문자로 변형하여 만들어준다.

[('standardscaler', StandardScaler()), ('svc', SVC())]

In [19]:
pipeline_svc = Pipeline([('scaler',StandardScaler()),('svc',SVC(probability=True))],verbose=True)
# pipeline_svc.steps # class 이름 소문자로 변형하여 만들어준다.

In [20]:
pipeline_svc.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scaler', 'svc', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'svc__C', 'svc__break_ties', 'svc__cache_size', 'svc__class_weight', 'svc__coef0', 'svc__decision_function_shape', 'svc__degree', 'svc__gamma', 'svc__kernel', 'svc__max_iter', 'svc__probability', 'svc__random_state', 'svc__shrinking', 'svc__tol', 'svc__verbose'])

In [21]:
%%time
param_grid = {
    'svc__kernel':['linear'],
    'svc__C' : [0.1,1,10],
    'svc__gamma':[0.001,0.01,0.1,1,10]
}


gs_svc = GridSearchCV(pipeline_svc,
                   param_grid = param_grid,
                   scoring = 'accuracy',
                   cv = [3,5,7,9,10],
                   n_jobs=-1)
gs_svc.fit(X_train, y_train)
pred_train_svc = gs_svc.predict(X_train)
pred_test_svc = gs_svc.predict(X_test)

accuracy_score(y_train, pred_train_svc), accuracy_score(y_test, pred_test_svc)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svc, total=  45.2s


(0.91056, 0.91456)

In [22]:
pd.DataFrame(gs_svc.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__gamma,param_svc__kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
39,69.704188,1.858100,4.618594,0.520597,1,10,linear,"{'svc__C': 1, 'svc__gamma': 10, 'svc__kernel':...",0.91232,0.90952,0.90976,0.910533,0.001267,1
37,83.662145,3.182245,6.259459,0.472571,1,1,linear,"{'svc__C': 1, 'svc__gamma': 1, 'svc__kernel': ...",0.91232,0.90952,0.90976,0.910533,0.001267,1
35,87.129672,0.784211,7.554821,0.164320,1,0.1,linear,"{'svc__C': 1, 'svc__gamma': 0.1, 'svc__kernel'...",0.91232,0.90952,0.90976,0.910533,0.001267,1
33,92.865383,1.653301,9.930054,1.218309,1,0.01,linear,"{'svc__C': 1, 'svc__gamma': 0.01, 'svc__kernel...",0.91232,0.90952,0.90976,0.910533,0.001267,1
31,95.140699,3.601978,10.043087,1.488911,1,0.001,linear,"{'svc__C': 1, 'svc__gamma': 0.001, 'svc__kerne...",0.91232,0.90952,0.90976,0.910533,0.001267,1
29,51.151530,0.987759,10.290848,0.164836,0.1,10,linear,"{'svc__C': 0.1, 'svc__gamma': 10, 'svc__kernel...",0.91240,0.90952,0.90896,0.910293,0.001507,6
27,57.751902,0.873750,8.789152,0.085460,0.1,1,linear,"{'svc__C': 0.1, 'svc__gamma': 1, 'svc__kernel'...",0.91240,0.90952,0.90896,0.910293,0.001507,6
25,55.912454,0.927015,9.604067,0.093562,0.1,0.1,linear,"{'svc__C': 0.1, 'svc__gamma': 0.1, 'svc__kerne...",0.91240,0.90952,0.90896,0.910293,0.001507,6
23,48.702349,3.762267,9.042430,0.117267,0.1,0.01,linear,"{'svc__C': 0.1, 'svc__gamma': 0.01, 'svc__kern...",0.91240,0.90952,0.90896,0.910293,0.001507,6
21,54.581265,0.976291,9.481182,0.609214,0.1,0.001,linear,"{'svc__C': 0.1, 'svc__gamma': 0.001, 'svc__ker...",0.91240,0.90952,0.90896,0.910293,0.001507,6


In [23]:
gs_svc_best_params = gs_svc.best_params_
gs_svc_best_params

{'svc__C': 1, 'svc__gamma': 0.001, 'svc__kernel': 'linear'}

In [24]:
gs_svc_best_etimator = gs_svc.best_estimator_
gs_svc_best_etimator

Pipeline(steps=[('scaler', StandardScaler()),
                ('svc', SVC(C=1, gamma=0.001, kernel='linear'))],
         verbose=True)

In [25]:
pred_train_gs_svc = gs_svc.predict(X_train)
pred_test_gs_svc = gs_svc.predict(X_test)

accuracy_score(y_train, pred_train_gs_svc), accuracy_score(y_test, pred_test_gs_svc)

(0.91056, 0.91456)

In [69]:
pipeline_svc_proba = Pipeline([('scaler',StandardScaler()),('svc',SVC(probability=True))],verbose=True)
pipeline_svc.steps

[('scaler', StandardScaler()), ('svc', SVC())]

In [70]:
pipeline_svc_proba.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scaler', 'svc', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'svc__C', 'svc__break_ties', 'svc__cache_size', 'svc__class_weight', 'svc__coef0', 'svc__decision_function_shape', 'svc__degree', 'svc__gamma', 'svc__kernel', 'svc__max_iter', 'svc__probability', 'svc__random_state', 'svc__shrinking', 'svc__tol', 'svc__verbose'])

In [71]:
%%time
param_grid = {
    'svc__kernel':['rbf','linear'],
    'svc__C' : [0.001,0.01,0.1,1],
    'svc__gamma':[0.001,0.01,0.1,1,10]
}


gs_svc_proba = GridSearchCV(pipeline_svc_proba,
                   param_grid = param_grid,
                   scoring = 'accuracy',
                   cv = 3,
                   n_jobs=-1)

gs_svc_proba.fit(X_train, y_train)
pred_train_svc = gs_svc_proba.predict(X_train)
pred_test_svc = gs_svc_proba.predict(X_test)

accuracy_score(y_train, pred_train_svc), accuracy_score(y_test, pred_test_svc)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svc, total= 4.2min


(0.91056, 0.91456)

In [83]:
gs_svc_cv_proba_result_df = pd.DataFrame(gs_svc_proba.cv_results_).sort_values('rank_test_score')
gs_svc_cv_proba_result_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__gamma,param_svc__kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
39,540.296725,11.968943,9.089273,0.783610,1,10,linear,"{'svc__C': 1, 'svc__gamma': 10, 'svc__kernel':...",0.91232,0.90952,0.90976,0.910533,0.001267,1
37,618.577237,15.202040,12.017666,0.936492,1,1,linear,"{'svc__C': 1, 'svc__gamma': 1, 'svc__kernel': ...",0.91232,0.90952,0.90976,0.910533,0.001267,1
35,639.040573,3.375926,13.911693,0.118174,1,0.1,linear,"{'svc__C': 1, 'svc__gamma': 0.1, 'svc__kernel'...",0.91232,0.90952,0.90976,0.910533,0.001267,1
33,639.291482,5.729777,14.421190,0.322978,1,0.01,linear,"{'svc__C': 1, 'svc__gamma': 0.01, 'svc__kernel...",0.91232,0.90952,0.90976,0.910533,0.001267,1
31,642.821918,6.721708,13.763346,0.556232,1,0.001,linear,"{'svc__C': 1, 'svc__gamma': 0.001, 'svc__kerne...",0.91232,0.90952,0.90976,0.910533,0.001267,1
29,451.601592,4.473998,13.266748,0.235527,0.1,10,linear,"{'svc__C': 0.1, 'svc__gamma': 10, 'svc__kernel...",0.91240,0.90952,0.90896,0.910293,0.001507,6
27,444.293345,3.993192,14.014897,0.080623,0.1,1,linear,"{'svc__C': 0.1, 'svc__gamma': 1, 'svc__kernel'...",0.91240,0.90952,0.90896,0.910293,0.001507,6
25,440.321303,4.684698,14.469833,0.149469,0.1,0.1,linear,"{'svc__C': 0.1, 'svc__gamma': 0.1, 'svc__kerne...",0.91240,0.90952,0.90896,0.910293,0.001507,6
23,400.606501,11.498459,13.680892,0.501275,0.1,0.01,linear,"{'svc__C': 0.1, 'svc__gamma': 0.01, 'svc__kern...",0.91240,0.90952,0.90896,0.910293,0.001507,6
21,421.157875,4.484277,10.323520,0.327558,0.1,0.001,linear,"{'svc__C': 0.1, 'svc__gamma': 0.001, 'svc__ker...",0.91240,0.90952,0.90896,0.910293,0.001507,6


In [72]:
gs_svc_proba_best_params = gs_svc_proba.best_params_
gs_svc_proba_best_params

{'svc__C': 1, 'svc__gamma': 0.001, 'svc__kernel': 'linear'}

In [73]:
gs_svc_proba_best_etimator = gs_svc_proba.best_estimator_
gs_svc_proba_best_etimator

Pipeline(steps=[('scaler', StandardScaler()),
                ('svc',
                 SVC(C=1, gamma=0.001, kernel='linear', probability=True))],
         verbose=True)

In [74]:
pred_train_gs_svc_proba = gs_svc_proba.predict(X_train)
pred_test_gs_svc_proba = gs_svc_proba.predict(X_test)

accuracy_score(y_train, pred_train_gs_svc_proba), accuracy_score(y_test, pred_test_gs_svc_proba)

(0.91056, 0.91456)

#### knn

In [26]:
pipeline_knn = Pipeline([('scaler',StandardScaler()),('knn',KNeighborsClassifier())],verbose=True)

In [27]:
pipeline_knn.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scaler', 'knn', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'knn__algorithm', 'knn__leaf_size', 'knn__metric', 'knn__metric_params', 'knn__n_jobs', 'knn__n_neighbors', 'knn__p', 'knn__weights'])

In [34]:
%%time
param_grid = {
    'knn__n_neighbors' : range(1,20),
}
gs_knn = GridSearchCV(pipeline_knn,
                   param_grid = param_grid,
                   scoring = 'accuracy',
                   cv = 3,
                   n_jobs=-1)
gs_knn.fit(X_train,y_train)
pred_train_knn = gs_knn.predict(X_train)
pred_test_knn = gs_knn.predict(X_test)

accuracy_score(y_train, pred_train_knn), accuracy_score(y_test, pred_test_knn)

(0.8814933333333334, 0.8648)

In [ ]:
pd.DataFrame(gs_knn.cv_results_).sort_values('rank_test_score')

In [60]:
gs_knn_best_estimator = gs_knn.best_estimator_
gs_knn_best_estimator

Pipeline(steps=[('scaler', StandardScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=15))])

In [59]:
gs_knn_best_params = gs_knn.best_params_
gs_knn_best_params

{'knn__n_neighbors': 15}

#### DecisionTreeClassifier()

In [35]:
tree = DecisionTreeClassifier()

param_grid={
    'max_depth':range(2,15),
    'min_samples_leaf':[100,500,1000,2000],
    'max_leaf_nodes':[5, 10, 20, 30],
    'random_state' :[1]
}
gs_tree = GridSearchCV(tree,
                      param_grid = param_grid, 
                      scoring = 'accuracy',                          
                      cv= 5,
                      n_jobs = -1)

In [36]:
%%time
gs_tree.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(2, 15),
                         'max_leaf_nodes': [5, 10, 20, 30],
                         'min_samples_leaf': [100, 500, 1000, 2000],
                         'random_state': [1]},
             scoring='accuracy')

In [37]:
pred_train_tree = gs_tree.predict(X_train)
pred_test_tree = gs_tree.predict(X_test)
accuracy_score(y_train,pred_train_tree), accuracy_score(y_test, pred_test_tree)

(0.9209066666666667, 0.92424)

In [38]:
best_tree_estimator = gs_tree.best_estimator_
best_tree_estimator

DecisionTreeClassifier(max_depth=7, max_leaf_nodes=30, min_samples_leaf=100,
                       random_state=1)

In [39]:
gs_tree_best_params = gs_tree.best_params_
gs_tree_best_params

{'max_depth': 7,
 'max_leaf_nodes': 30,
 'min_samples_leaf': 100,
 'random_state': 1}

In [40]:
best_tree_fi = best_tree_estimator.feature_importances_
best_tree_fi

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01172205, 0.        , 0.        , 0.        ,
       0.50684409, 0.        , 0.        , 0.01064892, 0.        ,
       0.        , 0.        , 0.47078495, 0.        , 0.        ,
       0.        ])

In [41]:
best_tree_fi_s = pd.Series(best_tree_fi, index=X_train.columns)
best_tree_fi_s.sort_values(ascending=False)

Speed_first          0.506844
Speed_second         0.470785
Attack_first         0.011722
Attack_second        0.010649
Generation_first     0.000000
Total_first          0.000000
Generation_second    0.000000
Sp. Def_second       0.000000
Sp. Atk_second       0.000000
Defense_second       0.000000
HP_second            0.000000
#_x                  0.000000
First_pokemon        0.000000
Sp. Def_first        0.000000
Sp. Atk_first        0.000000
Defense_first        0.000000
HP_first             0.000000
Legendary_second     0.000000
Legendary_first      0.000000
Second_pokemon       0.000000
Total_second         0.000000
dtype: float64

In [42]:
print(classification_report(y_train, pred_train_tree))

              precision    recall  f1-score   support

           1       0.91      0.93      0.92     17701
           2       0.93      0.92      0.92     19799

    accuracy                           0.92     37500
   macro avg       0.92      0.92      0.92     37500
weighted avg       0.92      0.92      0.92     37500



In [43]:
print(classification_report(y_test, pred_test_tree))

              precision    recall  f1-score   support

           1       0.92      0.92      0.92      5900
           2       0.93      0.92      0.93      6600

    accuracy                           0.92     12500
   macro avg       0.92      0.92      0.92     12500
weighted avg       0.92      0.92      0.92     12500



#### RandomForest()

In [44]:
param_grid={
    'max_depth':range(2,15),
    'n_estimators': [50, 100, ], #150, 200, 300
    'max_leaf_nodes':[5, 10, 20, 30],
    'random_state' :[1]
}
rf = RandomForestClassifier(random_state=1)
gs_rf = GridSearchCV(rf,
                    param_grid=param_grid,
                    scoring = 'accuracy',
                    cv = 3 ,
                    n_jobs=-1)

In [45]:
%%time
gs_rf.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=1), n_jobs=-1,
             param_grid={'max_depth': range(2, 15),
                         'max_leaf_nodes': [5, 10, 20, 30],
                         'n_estimators': [50, 100], 'random_state': [1]},
             scoring='accuracy')

In [46]:
pred_train_rf = gs_rf.predict(X_train)
pred_test_rf = gs_rf.predict(X_test)
accuracy_score(y_train,pred_train_rf), accuracy_score(y_test,pred_test_rf)

(0.8904533333333333, 0.88784)

In [47]:
best_rf_estimator = gs_rf.best_estimator_
best_rf_estimator

RandomForestClassifier(max_depth=12, max_leaf_nodes=30, random_state=1)

In [48]:
best_rf_fi = best_rf_estimator.feature_importances_
best_rf_fi

array([8.05544872e-04, 5.30560712e-04, 2.82480834e-04, 6.29535090e-03,
       6.68903504e-03, 1.15469736e-02, 3.66874708e-02, 8.25380511e-03,
       2.14160616e-02, 4.91549309e-03, 3.47774395e-01, 1.41013076e-04,
       7.79286810e-03, 2.59373903e-02, 7.61380849e-03, 2.47543713e-02,
       8.23237734e-03, 3.03684074e-01, 5.54941064e-05, 9.11345063e-02,
       8.54569247e-02])

In [49]:
best_rf_fi_s = pd.Series(best_rf_fi, index=X_train.columns)
best_rf_fi_s.sort_values(ascending=False)

Speed_first          0.347774
Speed_second         0.303684
Total_first          0.091135
Total_second         0.085457
Attack_first         0.036687
Attack_second        0.025937
Sp. Atk_second       0.024754
Sp. Atk_first        0.021416
HP_first             0.011547
Defense_first        0.008254
Sp. Def_second       0.008232
HP_second            0.007793
Defense_second       0.007614
Legendary_second     0.006689
Legendary_first      0.006295
Sp. Def_first        0.004915
#_x                  0.000806
First_pokemon        0.000531
Second_pokemon       0.000282
Generation_first     0.000141
Generation_second    0.000055
dtype: float64

#### XGBClassifier()

In [50]:
xgb = XGBClassifier(random_state=1,
                    objective='binary:logistic',
                   )

param_grid_xgb = {
    'n_estimators' : [50,100,150, 200, 250], # 300,350,400,450,500
#     "learning_rate": [0.001, 0.01, 0.1, 0.3, 0.5],
#     "max_depth": [5, 10, 15, 20, 30]
#     'subsample' : [0.5,0.6,0.9],
#     'colsample_bytree':[0.5,0.6,0.9]
}

In [51]:
# grid search
gs_xgb = GridSearchCV(xgb, 
                           param_grid = param_grid_xgb, 
                           scoring=["accuracy",'recall','precision'], 
                           refit="accuracy", 
                           cv = 10, 
                           n_jobs=-1
                          )

In [53]:
%%time
gs_xgb.fit(X_train, y_train)

/Users/jonghae/venv-tf/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:29:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=1,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, v

In [54]:
# predict
pred_train_xgb = gs_xgb.predict(X_train)
pred_test_xgb = gs_xgb.predict(X_test)

In [55]:
accuracy_score(y_train, pred_train_xgb), accuracy_score(y_test, pred_test_xgb), 

(0.9951733333333334, 0.94896)

In [56]:
df_xgb = pd.DataFrame(gs_xgb.cv_results_)
df_xgb

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,...,split3_test_precision,split4_test_precision,split5_test_precision,split6_test_precision,split7_test_precision,split8_test_precision,split9_test_precision,mean_test_precision,std_test_precision,rank_test_precision
0,7.063584,0.101211,0.024956,0.002864,50,{'n_estimators': 50},0.947200,0.945333,0.952267,0.944000,...,0.930939,0.938513,0.941769,0.920765,0.935919,0.934481,0.933811,0.934285,0.005686,5
1,17.353109,1.606338,0.033459,0.003360,100,{'n_estimators': 100},0.948800,0.948533,0.952267,0.942933,...,0.932222,0.937465,0.941077,0.922192,0.940476,0.934928,0.936429,0.936271,0.005559,4
2,30.990564,0.946906,0.035464,0.004739,150,{'n_estimators': 150},0.948533,0.950667,0.953867,0.944000,...,0.934783,0.935484,0.943852,0.925439,0.940943,0.935862,0.937465,0.937663,0.005572,3
3,46.033437,0.694436,0.034151,0.006444,200,{'n_estimators': 200},0.949067,0.951200,0.950667,0.944267,...,0.933852,0.939276,0.945763,0.925865,0.944824,0.937535,0.941242,0.939230,0.005647,2
4,42.612314,8.978964,0.025535,0.005926,250,{'n_estimators': 250},0.950933,0.951467,0.951200,0.944000,...,0.934298,0.939799,0.944225,0.927512,0.944792,0.939106,0.943302,0.939926,0.005082,1


In [57]:
# best param
best_params_xgb = gs_xgb.best_params_
best_params_xgb

{'n_estimators': 250}

In [58]:
# best model
best_estimator_xgb = gs_xgb.best_estimator_
best_estimator_xgb, type(best_estimator_xgb)

(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.300000012, max_delta_step=0, max_depth=6,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=250, n_jobs=16, num_parallel_tree=1, random_state=1,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
               tree_method='exact', validate_parameters=1, verbosity=None),
 xgboost.sklearn.XGBClassifier)

### Ensemble
- voting

In [75]:
estimators = [('svc', gs_svc_proba_best_etimator ),
              ('knn', gs_knn_best_estimator),
              ('Decision tree', best_tree_estimator),
              ('random forest', best_rf_estimator),
              ('XGBoost', best_estimator_xgb)]

In [76]:
voting = VotingClassifier(estimators,
                          voting='soft',
                         )

In [77]:
def print_metrics(y, y_pred, title=None):
    if title: #title != None
        print(title)
    metrics = f'정확도:{accuracy_score(y, y_pred)}, 재현율:{recall_score(y, y_pred)}, 정밀도:{precision_score(y, y_pred)}, f1점수:{f1_score(y, y_pred)}'
    print(metrics)

In [78]:
%%time
voting.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svc, total= 4.4min


/Users/jonghae/venv-tf/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:49:57] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


VotingClassifier(estimators=[('svc',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('svc',
                                               SVC(C=1, gamma=0.001,
                                                   kernel='linear',
                                                   probability=True))],
                                       verbose=True)),
                             ('knn',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('knn',
                                               KNeighborsClassifier(n_neighbors=15))])),
                             ('Decision tree',
                              DecisionTreeClassifier(max_depth=7,
                                                     max_leaf_nodes=30,
                                                     min_samples_leaf=100,
                                                     

In [79]:
pred_train = voting.predict(X_train)
pred_test = voting.predict(X_test)

In [82]:
print_metrics(y_train, pred_train, "Train 데이터셋-Soft Voting")
print_metrics(y_test, pred_test, "val 데이터셋-Soft Voting")

Train 데이터셋-Soft Voting
정확도:0.9407733333333334, 재현율:0.9450313541607819, 정밀도:0.9305740987983978, f1점수:0.9377470078762227
val 데이터셋-Soft Voting
정확도:0.93736, 재현율:0.9364406779661016, 정밀도:0.9312320916905444, f1점수:0.933829121947097
